# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Boston Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import the Boston data set using `load_boston()`. We won't bother to preprocess the data in this lab. If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [51]:
regression = LinearRegression()
boston = load_boston()

In [107]:
#My code
boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
X = boston_features
y = pd.DataFrame(boston.target, columns=['Target'])

In [114]:
#their code
y = pd.DataFrame(boston.target,columns = ['target'])
df = pd.DataFrame(boston.data, columns = boston.feature_names)
all_data = pd.concat([y,df], axis = 1)

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, df, y, scoring='r2', cv=crossvalidation))
baseline

0.7190106820189477

Create a baseline model which includes all the variables in the Boston housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [108]:
## my code here
cross_validation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression,X,y,scoring='r2', cv=cross_validation))

In [109]:
baseline

0.7190106820189477

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

In [97]:
from itertools import combinations
combinations = list(combinations(boston.feature_names, 2))

In [111]:
#mycode
interaction_combos = {}
data = X
for combo in combinations:    
    interaction = data[combo[0]] * data[combo[1]]
    #interaction = boston_features[combinations[0][0]] * boston_features[combinations[0][1]]
    interaction_name = combo[0] + "_"+ combo[1]
    #interaction_name = combinations[0][0] + "_"+ combinations[0][1]
    data[interaction_name] = interaction
    cross_validation = KFold(n_splits=10, shuffle=True, random_state=1)
    baseline = np.mean(cross_val_score(regression,data,y,scoring='r2', cv=cross_validation))
    interaction_combos[interaction_name] = baseline

In [112]:
## code to find top 3 interactions by R^2 value here
[[k, v] for k, v in sorted(interaction_combos.items(), key=lambda item: item[1], reverse=True)][:3]

[['TAX_PTRATIO', 0.8532317331733953],
 ['TAX_B', 0.8531244563973182],
 ['TAX_LSTAT', 0.8528562547505093]]

In [115]:
#their code
interactions = []
data = df.copy()
for comb in combinations:
    data['interaction'] = data[comb[0]] * data[comb[1]]
    score = np.mean(cross_val_score(regression, data, y, scoring='r2', cv=crossvalidation))
    if score > baseline: interactions.append((comb[0], comb[1], round(score,3)))
            
print('Top 3 interactions: %s' %sorted(interactions, key=lambda inter: inter[2], reverse=True)[:5])

Top 3 interactions: [('RM', 'LSTAT', 0.786), ('RM', 'TAX', 0.775), ('RM', 'RAD', 0.768), ('RM', 'PTRATIO', 0.763), ('INDUS', 'RM', 0.758)]


In [116]:
#their code with my X as a copy
interactions = []
data = X.copy()
for comb in combinations:
    data['interaction'] = data[comb[0]] * data[comb[1]]
    score = np.mean(cross_val_score(regression, data, y, scoring='r2', cv=cross_validation))
    if score > baseline: interactions.append((comb[0], comb[1], round(score,3)))

In [117]:
print('Top 3 interactions: %s' %sorted(interactions, key=lambda inter: inter[2], reverse=True)[:5])

Top 3 interactions: [('CRIM', 'ZN', 0.846), ('CRIM', 'INDUS', 0.846), ('CRIM', 'CHAS', 0.846), ('CRIM', 'NOX', 0.846), ('CRIM', 'RM', 0.846)]


## Look at the top 3 interactions: "RM" as a confounding factor

The top three interactions seem to involve "RM", the number of rooms as a confounding variable for all of them. Let's have a look at interaction plots for all three of them. This exercise will involve:

- Splitting the data up in 3 groups: one for houses with a few rooms, one for houses with a "medium" amount of rooms, one for a high amount of rooms 
- Create a function `build_interaction_rm()`. This function takes an argument `varname` (which can be set equal to the column name as a string) and a column `description` (which describes the variable or varname, to be included on the x-axis of the plot). The function outputs a plot that uses "RM" as a confounding factor. Each plot should have three regression lines, one for each level of "RM"  

The data has been split into high, medium, and low number of rooms for you.

In [ ]:
rm = np.asarray(df[['RM']]).reshape(len(df[['RM']]))

In [ ]:
high_rm = all_data[rm > np.percentile(rm, 67)]
med_rm = all_data[(rm > np.percentile(rm, 33)) & (rm <= np.percentile(rm, 67))]
low_rm = all_data[rm <= np.percentile(rm, 33)]

Create `build_interaction_rm()`.

In [ ]:
def build_interaction_rm(varname, description):
    pass

Next, use `build_interaction_rm()` with the three variables that came out with the highest effect on $R^2$. 

In [ ]:
# first plot

In [ ]:
# second plot

In [ ]:
# third plot

## Build a final model including all three interactions at once

Use 10-fold cross-validation to build a model using all the above interactions. 

In [ ]:
# code here

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if all these interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.